In [1]:
import neurotorch as nt

In [2]:
checkpoint_folder = f"./checkpoints/network"
checkpoint_manager = nt.CheckpointManager(checkpoint_folder)
print(checkpoint_manager)

CheckpointManager<0>: (priority=100, save_state=False, load_state=False, )


In [3]:
network = nt.Sequential(layers=[nt.Linear(10, 10)], checkpoint_folder=checkpoint_folder).build()
print(network)

Sequential(
  (_to_device_transform): ToDevice(cuda, async=True)
  (input_layers): ModuleDict()
  (hidden_layers): ModuleList()
  (output_layers): ModuleDict(
    (output_0): Linear<output_0>(10->10)@cuda
  )
  (input_transform): ModuleDict(
    (output_0): Sequential(
      (0): CallableToModuleWrapper(Compose(
          ToTensor()
      ))
      (1): ToDevice(cuda, async=True)
    )
  )
  (output_transform): ModuleDict(
    (output_0): IdentityTransform()
  )
)


In [4]:
checkpoint_manager.checkpoints_meta_path

'./checkpoints/network/network-checkpoints.json'

In [5]:
checkpoint_manager.save_checkpoint(itr=0, itr_metrics={}, state_dict=network.state_dict())

'./checkpoints/network\\network-itr0.pth'

In [6]:
network.get_layer().forward_weights.data.zero_()

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [7]:
network.load_checkpoint(checkpoints_meta_path=checkpoint_manager.checkpoints_meta_path, load_checkpoint_mode=nt.LoadCheckpointMode.LAST_ITR)

{'itr': 0,
 'model_state_dict': OrderedDict([('output_layers.output_0.bias_weights',
               tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')),
              ('output_layers.output_0._forward_weights',
               tensor([[-7.3540e-01,  4.3947e-01,  4.3454e-04, -1.5365e-01,  2.7009e-01,
                         1.1282e-01,  4.2071e-02, -2.2711e-01,  3.6099e-01,  1.5909e-01],
                       [-4.6935e-02, -1.7355e-01,  2.8171e-01,  3.0744e-01, -3.5100e-01,
                         1.7603e-02,  6.6100e-02,  1.3925e-01, -7.1727e-02, -9.7251e-03],
                       [-7.4916e-02, -3.1412e-02,  1.8288e-01,  4.4510e-01,  9.3525e-02,
                         2.9784e-02,  1.2521e-01, -7.2491e-02, -4.5314e-01,  3.9042e-01],
                       [-1.9225e-01,  1.7739e-01,  2.8031e-01,  2.1685e-01,  1.9586e-01,
                        -1.3653e-01,  7.9019e-02,  1.3299e-01,  3.2456e-01, -4.5222e-01],
                       [-8.3174e-02, -3.9672e-01,  2.9031e

In [8]:
network.get_layer().forward_weights

Parameter containing:
tensor([[-7.3540e-01,  4.3947e-01,  4.3454e-04, -1.5365e-01,  2.7009e-01,
          1.1282e-01,  4.2071e-02, -2.2711e-01,  3.6099e-01,  1.5909e-01],
        [-4.6935e-02, -1.7355e-01,  2.8171e-01,  3.0744e-01, -3.5100e-01,
          1.7603e-02,  6.6100e-02,  1.3925e-01, -7.1727e-02, -9.7251e-03],
        [-7.4916e-02, -3.1412e-02,  1.8288e-01,  4.4510e-01,  9.3525e-02,
          2.9784e-02,  1.2521e-01, -7.2491e-02, -4.5314e-01,  3.9042e-01],
        [-1.9225e-01,  1.7739e-01,  2.8031e-01,  2.1685e-01,  1.9586e-01,
         -1.3653e-01,  7.9019e-02,  1.3299e-01,  3.2456e-01, -4.5222e-01],
        [-8.3174e-02, -3.9672e-01,  2.9031e-01, -1.6875e-01,  4.6306e-02,
          4.1986e-01, -1.4235e-01,  2.3946e-02,  2.3502e-01, -4.7609e-01],
        [ 1.9356e-02, -2.1739e-02,  1.7779e-01, -3.4789e-02,  4.4785e-02,
          8.8605e-02,  2.9104e-01,  1.5570e-01, -7.0589e-01,  2.3611e-01],
        [-3.1387e-01, -4.6288e-01,  3.2183e-02,  5.0033e-02,  1.3759e-02,
         -